# Small Experiments Pertaining to Shaders for Scientific Visualisation
The contents of this notebook is neither documentation nor does it serve as permanent unit tests for the project. It's purely kept as a record of some of the experimentation done while working on this project.

### Custom HTML Output in Jupyter

In [2]:
class CustomHTMLObject:
    def __init__(self, name: str, colour: str):
        """
        
        :param name: The name of the object
        :param colour: The hex colour code of the object (eg: "#112233")
        """
        self.name = name
        self.colour = colour
        
    def _repr_html_(self) -> str:
        """
        Hooks into Jupyter notebook rich display system, which calls _repr_html_ by
        default if an object is returned at the end of a cell.
        """
        return f"""
        <div class="custom-html-object">
            <div style="color: {self.colour};font-size: large;">{self.name}</div>
        </div>"""
    
# Now return a CustomHTMLObject to the cell
CustomHTMLObject("Custom HTML is cool!", "#ff2010")

In [1]:
%pip install moderngl

     ---------------------------------------- 0.0/106.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/106.8 kB ? eta -:--:--
     -------------------------------------- 106.8/106.8 kB 1.5 MB/s eta 0:00:00
  Obtaining dependency information for glcontext<3,>=2.3.6 from https://files.pythonhosted.org/packages/75/65/a4cd375ff65099f9b6203a960337c24d21d12162107b0fe4f55dd617d2e8/glcontext-2.4.0-cp310-cp310-win_amd64.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from PIL import Image
import moderngl

In [11]:
# Setup the OpenGL context
ctx = moderngl.create_context(standalone=True)
print(f"Got OpenGL context:\n\tGL_VENDOR={ctx.info['GL_VENDOR']}\n\tGL_RENDERER={ctx.info['GL_RENDERER']}\n\tGL_VERSION={ctx.info['GL_VERSION']}")
resolution = (20000, 20000)
fbo = ctx.simple_framebuffer(resolution, components=4)
fbo.use()

# Create a vertex buffer
vertices = np.array([
   # X      Y      R    G    B
    -1.0,  -1.0,   1.0, 0.0, 0.0,
     1.0,  -1.0,   0.0, 1.0, 0.0,
     0.0,   1.0,   0.0, 0.0, 1.0],
    dtype='f4',
)

prog = ctx.program(vertex_shader="""
#version 330
in vec2 in_vert;
in vec3 in_color;
out vec3 color;
out vec2 position;
void main() {
    gl_Position = vec4(in_vert, 0.0, 1.0);
    color = in_color;
    position = in_vert*0.5+0.5;
}
""",
    fragment_shader="""
#version 330
out vec4 fragColor;
in vec3 color;
in vec2 position;

uniform vec2 iResolution;

float amod(float x, float y)
{
    return x - y * floor(x/y);
}

vec4 mainImage(in vec2 fragCoord)
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xy;

    float coord = floor(fragCoord.x) + floor(fragCoord.y/10.);
    //vec3 col = vec3(amod(coord, 16.)>=8.?1.:0., amod(coord, 32.)>=16.?1.:0., amod(coord, 64.)>=32.?1.:0.);
    //col = amod(coord, 128.)>64.?(col*0.3333+.3333):col;

    vec3 col = vec3(amod(coord, 64.) >= 56. ? 1. : 0.,
                    amod(coord + 16., 64.) >= 56. ? 1. : 0.,
                    amod(coord + 32., 64.) >= 56. ? 1. : 0.);
    col += amod(coord + 48., 64.) >= 56. ? 1. : 0.;
    col = amod(coord, 128.) > 64. ? (col * 0.3333 + .3333) : col;

    // Output to screen
    return vec4(col,1.0);
}

void main() {
    fragColor = mainImage(position * iResolution) + vec4(color, 1.0)*0.01;
    //fragColor = vec4(color, 1.0);
}
""",
)

# Set uniforms
prog["iResolution"].value = resolution

# Assign buffers and render
vao = ctx.simple_vertex_array(prog, ctx.buffer(vertices), 'in_vert', 'in_color')
vao.render(mode=moderngl.TRIANGLES)

# Save output to png
print("Saving image...")
image = Image.frombytes('RGBA', resolution, fbo.read(components=4))
image = image.transpose(Image.FLIP_TOP_BOTTOM)
image.save('openglHeadlessExample.png', format='png')
print(f"Saved to 'openglHeadlessExample.png'!")

Got OpenGL context:
	GL_VENDOR=NVIDIA Corporation
	GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
	GL_VERSION=3.3.0 NVIDIA 536.23
Saving image...
Saved to 'openglHeadlessExample.png'!
